# Create Graph

In [1]:
import osmnx as ox
import networkx as nx
import shapely

# Step 1: Create a Street Network Graph
place_name = "Lustenau, Vorarlberg, Austria"
G = ox.graph_from_place(place_name, network_type='drive', simplify=True)
G_proj = G.to_undirected()
ox.basic_stats(G_proj)

{'n': 687,
 'm': 877,
 'k_avg': 2.5531295487627363,
 'edge_length_total': 113441.661,
 'edge_length_avg': 129.35195096921322,
 'streets_per_node_avg': 2.5705967976710333,
 'streets_per_node_counts': {0: 0, 1: 165, 2: 3, 3: 481, 4: 38},
 'streets_per_node_proportions': {0: 0.0,
  1: 0.24017467248908297,
  2: 0.004366812227074236,
  3: 0.7001455604075691,
  4: 0.055312954876273655},
 'intersection_count': 522,
 'street_length_total': 113178.752,
 'street_segment_count': 876,
 'street_length_avg': 129.1994885844749,
 'circuity_avg': 1.0429455363347429,
 'self_loop_proportion': 0.001141552511415525}

# Eulerize

In [2]:
G_proj_euler = nx.eulerize(G_proj)
nx.is_eulerian(G_proj_euler)

True

# Get Nodes and Edges

In [3]:
lustenau_nodes_df, lustenau_edges_df = ox.graph_to_gdfs(G_proj)
lustenau_nodes_df_euler, lustenau_edges_df_euler = ox.graph_to_gdfs(G_proj_euler)

# get newly added edges

In [4]:
import pandas as pd
non_euler_index = lustenau_edges_df.index
euler_index = lustenau_edges_df_euler.index
diff_index = euler_index.drop(non_euler_index)
lustenau_edges_df_euler.loc[diff_index]

osmid oneway lanes  ref name highway maxspeed  \
u          v           key                                                 
2113259    915529984   1     NaN    NaN   NaN  NaN  NaN     NaN      NaN   
           323321791   1     NaN    NaN   NaN  NaN  NaN     NaN      NaN   
2113269    1836257402  1     NaN    NaN   NaN  NaN  NaN     NaN      NaN   
2113277    2060864001  1     NaN    NaN   NaN  NaN  NaN     NaN      NaN   
11666841   2859477962  1     NaN    NaN   NaN  NaN  NaN     NaN      NaN   
...                          ...    ...   ...  ...  ...     ...      ...   
3737747548 3737747551  1     NaN    NaN   NaN  NaN  NaN     NaN      NaN   
4376974014 5873370851  1     NaN    NaN   NaN  NaN  NaN     NaN      NaN   
           4376974015  1     NaN    NaN   NaN  NaN  NaN     NaN      NaN   
4519304493 7717722801  1     NaN    NaN   NaN  NaN  NaN     NaN      NaN   
4622014941 11064350904 1     NaN    NaN   NaN  NaN  NaN     NaN      NaN   

                           reversed  length bridge  \
u          v           key                           
2113259    915529984   1        NaN     NaN    NaN   
           323321791   1        NaN     NaN    NaN   
2113269    1836257402  1        NaN     NaN    NaN   
2113277    2060864001  1        NaN     NaN    NaN   
11666841   2859477962  1        NaN     NaN    NaN   
...                             ...     ...    ...   
3737747548 3737747551  1        NaN     NaN    NaN   
4376974014 5873370851  1        NaN     NaN    NaN   
           4376974015  1        NaN     NaN    NaN   
4519304493 7717722801  1        NaN     NaN    NaN   
4622014941 11064350904 1        NaN     NaN    NaN   

                                                                   geometry  \
u          v           key                                                    
2113259    915529984   1    LINESTRING (9.67179 47.44613, 9.67236 47.44655)   
           323321791   1    LINESTRING (9.67179 47.44613, 9.67429 47.44063)   
2113269    1836257402  1    LINESTRING (9.66317 47.44597, 9.66244 47.44562)   
2113277    2060864001  1    LINESTRING (9.66476 47.45052, 9.66477 47.45064)   
11666841   2859477962  1    LINESTRING (9.67844 47.39777, 9.67829 47.39772)   
...                                                                     ...   
3737747548 3737747551  1    LINESTRING (9.65907 47.40561, 9.65941 47.40624)   
4376974014 5873370851  1    LINESTRING (9.69213 47.39977, 9.67885 47.39801)   
           4376974015  1    LINESTRING (9.69213 47.39977, 9.69252 47.39977)   
4519304493 7717722801  1    LINESTRING (9.67230 47.41799, 9.67214 47.41782)   
4622014941 11064350904 1    LINESTRING (9.65778 47.42669, 9.65724 47.42678)   

                           junction width tunnel  
u          v           key                        
2113259    915529984   1        NaN   NaN    NaN  
           323321791   1        NaN   NaN    NaN  
2113269    1836257402  1        NaN   NaN    NaN  
2113277    2060864001  1        NaN   NaN    NaN  
11666841   2859477962  1        NaN   NaN    NaN  
...                             ...   ...    ...  
3737747548 3737747551  1        NaN   NaN    NaN  
4376974014 5873370851  1        NaN   NaN    NaN  
           4376974015  1        NaN   NaN    NaN  
4519304493 7717722801  1        NaN   NaN    NaN  
4622014941 11064350904 1        NaN   NaN    NaN  

[368 rows x 14 columns]

# define helper methods to find route in added 

In [6]:
from shapely.geometry import Point, LineString

def find_route_between_coordinates(graph, origin_point, destination_point):
    # get closes graph nodes to origin and destination
    origin_point_x = min(origin_point[0], origin_point[1])
    origin_point_y = max(origin_point[0], origin_point[1]) # TODO swalkner
    destination_point_x = min(origin_point[0], origin_point[1])
    destination_point_y = max(destination_point[0], destination_point[1]) # TODO swalkner
    orig_node = ox.distance.nearest_nodes(graph, origin_point_x, origin_point_y)
    destination_node = ox.distance.nearest_nodes(graph, destination_point_x, destination_point_y)
    route = nx.shortest_path(graph, orig_node, destination_node, weight='travel_time')
    if len(route) == 1:
        route.append(route[0])
    return list(map(lambda x: graph.nodes[x], route))

def point_from_dict(dict):
    return Point(dict['x'], dict['y'])

def linestring_from_dict_list(dict_list):
    return LineString(map(lambda x: point_from_dict(x), dict_list))
    
place_name = "Lustenau, Vorarlberg, Austria"
G_route = ox.graph_from_place(place_name, network_type='walk', simplify=False)

In [7]:
trying = lustenau_edges_df_euler.copy()
trying.loc[diff_index, 'geometry'] = trying.loc[diff_index].apply(lambda x: linestring_from_dict_list(find_route_between_coordinates(G_route, x['geometry'].coords[0], x['geometry'].coords[1])), axis=1)

/Users/stefan/anaconda3/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [8]:
def f(frame):
    xy = frame.geometry.xy
    longs = xy[0].tolist()
    lats = xy[1].tolist()
    return [list(z) for z in zip(lats, longs)]

trying_index = trying.index
trying.loc[trying_index, 'coords'] = trying.loc[trying_index].apply(f, axis=1)
trying[trying['name'] == "Hofsteigstraße"]

osmid oneway  \
u         v          key                                                       
265000679 323326723  0                                       72290425  False   
          323324424  0                        [1189585841, 259382364]  False   
265000683 323329803  0                                       72290416  False   
          323325679  0                                       72290416  False   
265000687 323326089  0                                       72290416  False   
          323323695  0                         [442923578, 281538486]  False   
323315525 506964404  0                                      281538486  False   
          506964408  0                                      281538486  False   
          323328465  0                                       29392921  False   
323315537 323329803  0                                       72290416  False   
          2855121200 0                                       72290416  False   
323317480 621490448  0                                      146627137  False   
          323325473  0                                      146627137  False   
323319632 4675036995 0                                       72290425  False   
          323326723  0                                       72290425  False   
323320821 323324424  0                                      259382364  False   
          2855121200 0                                      281549425  False   
323323695 506964388  0                                      281538486  False   
323325051 323325679  0                                       72290416  False   
          323326089  0                                       72290416  False   
323325473 323325637  0                                       37290631  False   
323325496 323325637  0                                       37290631  False   
          323325543  0                [37290632, 37290631, 442585679]  False   
323325543 323329860  0    [79317272, 321988913, 442585678, 442585679]  False   
          323325546  0                                     1136127798  False   
323325637 323325643  0                                       29392732  False   
323329812 621490448  0                                      146627137  False   
          506964408  0                                      281538486  False   
506964388 506964395  0                                      281538486  False   
506964395 1824807478 0                                      281538486  False   
506964404 1824807478 0                                      281538486  False   
506964407 506964408  0                                       41451812  False   

                         lanes  ref            name       highway  maxspeed  \
u         v          key                                                      
265000679 323326723  0       2  NaN  Hofsteigstraße  unclassified        40   
          323324424  0     NaN  NaN  Hofsteigstraße  unclassified        40   
265000683 323329803  0     NaN  NaN  Hofsteigstraße  unclassified        40   
          323325679  0     NaN  NaN  Hofsteigstraße  unclassified        40   
265000687 323326089  0     NaN  NaN  Hofsteigstraße  unclassified        40   
          323323695  0     NaN  NaN  Hofsteigstraße  unclassified        40   
323315525 506964404  0     NaN  NaN  Hofsteigstraße  unclassified        40   
          506964408  0     NaN  NaN  Hofsteigstraße  unclassified        40   
          323328465  0     NaN  NaN  Hofsteigstraße   residential        40   
323315537 323329803  0     NaN  NaN  Hofsteigstraße  unclassified        40   
          2855121200 0     NaN  NaN  Hofsteigstraße  unclassified        40   
323317480 621490448  0     NaN  NaN  Hofsteigstraße  unclassified        40   
          323325473  0     NaN  NaN  Hofsteigstraße  unclassified        40   
323319632 4675036995 0       2  NaN  Hofsteigstraße  unclassified        40   
          323326723  0       2  NaN  Hofsteigstraße  unclassified        40   


In [9]:
coords_x = []
coords_y = []

for temp_coord in trying['coords']:
    for coord in temp_coord:
        coords_x.append(coord[0])
        coords_y.append(coord[1])
        
len(coords_x)

9074

In [11]:
import networkx as nx
from itertools import combinations

# Step 2: Convert to an Edge Graph
edge_graph = nx.Graph()
for u, v in G_proj_euler.edges(data=False):
    edge_graph.add_node((u, v))  # Each edge is a node

for node1, node2 in combinations(edge_graph.nodes, 2):
    # Add edges between nodes that have a common endpoint in the original graph
    if node1[1] == node2[0] or node1[0] == node2[1] or node1[1] == node2[1] or node1[0] == node2[0]:
        edge_graph.add_edge(node1, node2)
tsp_path = nx.approximation.traveling_salesman_problem(edge_graph, cycle=False)
edge_graph

In [12]:
import gpxpy

waypoints = []

for node in tsp_path:
    first = node[0]
    second = node[1]
    waypoints.append(gpxpy.gpx.GPXWaypoint(latitude=G_proj_euler.nodes[first]['x'], longitude=G_proj_euler.nodes[first]['y'], name=""))
    waypoints.append(gpxpy.gpx.GPXWaypoint(latitude=G_proj_euler.nodes[second]['x'], longitude=G_proj_euler.nodes[second]['y'], name=""))
    
waypoints

[GPXWaypoint(9.671793, 47.4461269, name=''),
 GPXWaypoint(9.6967666, 47.4462899, name=''),
 GPXWaypoint(9.6817583, 47.4367789, name=''),
 GPXWaypoint(9.6967666, 47.4462899, name=''),
 GPXWaypoint(9.6817583, 47.4367789, name=''),
 GPXWaypoint(9.6818031, 47.4356428, name=''),
 GPXWaypoint(9.6789064, 47.436255, name=''),
 GPXWaypoint(9.6817583, 47.4367789, name=''),
 GPXWaypoint(9.678877, 47.4368997, name=''),
 GPXWaypoint(9.6789064, 47.436255, name=''),
 GPXWaypoint(9.6758505, 47.4364486, name=''),
 GPXWaypoint(9.678877, 47.4368997, name=''),
 GPXWaypoint(9.6760462, 47.4357401, name=''),
 GPXWaypoint(9.6758505, 47.4364486, name=''),
 GPXWaypoint(9.6753975, 47.4356188, name=''),
 GPXWaypoint(9.6760462, 47.4357401, name=''),
 GPXWaypoint(9.6753975, 47.4356188, name=''),
 GPXWaypoint(9.6749254, 47.4355287, name=''),
 GPXWaypoint(9.6753191, 47.4351698, name=''),
 GPXWaypoint(9.6749254, 47.4355287, name=''),
 GPXWaypoint(9.6759493, 47.4347372, name=''),
 GPXWaypoint(9.6753191, 47.4351698, nam

In [13]:
import gpxpy.gpx

gpx = gpxpy.gpx.GPX()

# Create first track in our GPX:
gpx_track = gpxpy.gpx.GPXTrack()
gpx.tracks.append(gpx_track)

# Create first segment in our GPX track:
gpx_segment = gpxpy.gpx.GPXTrackSegment()
gpx_track.segments.append(gpx_segment)

for waypoint in waypoints:
    gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(waypoint.longitude, waypoint.latitude))

print(gpx.to_xml())

<?xml version="1.0" encoding="UTF-8"?>
<gpx xmlns="http://www.topografix.com/GPX/1/1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.topografix.com/GPX/1/1 http://www.topografix.com/GPX/1/1/gpx.xsd" version="1.1" creator="gpx.py -- https://github.com/tkrajina/gpxpy">
  <trk>
    <trkseg>
      <trkpt lat="47.4461269" lon="9.671793">
      </trkpt>
      <trkpt lat="47.4462899" lon="9.6967666">
      </trkpt>
      <trkpt lat="47.4367789" lon="9.6817583">
      </trkpt>
      <trkpt lat="47.4462899" lon="9.6967666">
      </trkpt>
      <trkpt lat="47.4367789" lon="9.6817583">
      </trkpt>
      <trkpt lat="47.4356428" lon="9.6818031">
      </trkpt>
      <trkpt lat="47.436255" lon="9.6789064">
      </trkpt>
      <trkpt lat="47.4367789" lon="9.6817583">
      </trkpt>
      <trkpt lat="47.4368997" lon="9.678877">
      </trkpt>
      <trkpt lat="47.436255" lon="9.6789064">
      </trkpt>
      <trkpt lat="47.4364486" lon="9.6758505">
      </tr